In [1]:
%reset -f

In [2]:
import pandas as pd
## We use a Factory object in the ontobio library
from ontobio import OntologyFactory
## We use the standard python networkx library for pathfinding here
## This is easily extracted from an ontology object
from networkx import nx

In [3]:
outputDir = '../../nlp/data/nlp/output/'
outFile = 'runNER_Output.tsv'
inputDir = '../../nlp/data/input/'
inFile = 'biosamples-expanded.tsv'
#cols = ['BiosampleId', 'Biolink', 'BeginTerm', 'EndTerm', 'TokenizedTerm', 'PreferredTerm', 'CURIE', 'NaN1', 'S-Type', 'NaN2', 'UMLS_CUI']
runnerOutput = pd.read_csv(outputDir+outFile, sep='\t' )
runnerOutput = runnerOutput.drop(['START POSITION', 'END POSITION', 'ZONE','SENTENCE ID', 'ORIGIN', 'UMLS CUI', 'SENTENCE'], axis = 1)
runnerOutput = runnerOutput.drop_duplicates()
print(len(runnerOutput))
runnerOutput.head()


205959


,DOCUMENT ID,TYPE,MATCHED TERM,PREFERRED FORM,ENTITY ID
0,gold:Gb0101224,biolink:OntologyClass,Coal,coal,ENVO:02000091
1,gold:Gb0101224,biolink:OntologyClass,Coal Mine,coal mine,ENVO:00002169
4,gold:Gb0101224,biolink:ChemicalSubstance,water,water,CHEBI:15377
5,gold:Gb0101224,biolink:OntologyClass,water,liquid water,ENVO:00002006_SYNONYM
6,gold:Gb0101225,biolink:OntologyClass,Coal,coal,ENVO:02000091


In [4]:
%%time
## Get the HPO using default method (currently OntoBee SPARQL)
## This may take 5-10s the first time you run it; afterwards it is cached
ofa = OntologyFactory()
ontology = ofa.create('envo')
## The OWL version of ENVO (used here) has many interesting relationship types;
## for now we just care about is-a (subClassOf between named classes)
ont = ontology.subontology(relations=['subClassOf', 'PART_OF'])

CPU times: user 3.67 s, sys: 57.4 ms, total: 3.73 s
Wall time: 9.73 s


In [5]:
def getAncestors(entity):
    """
    Get all ancestors of entities
    """
    curieList = ont.search(entity)
    ancestorList = []
    for item in curieList:
        ancestorList.append(list(ont.ancestors(item)))
    
    return ancestorList

def getDescendants(entity):
    curieList = ont.search(entity)
    descendantList = []
    for item in curieList:
        descendantList.append(list(ont.descendants(item)))
    
    return descendantList
     

In [6]:
uniquePreferredTerms = runnerOutput['PREFERRED FORM'].unique()
uniquePreferredTermsDF = pd.DataFrame(uniquePreferredTerms, columns=['PREFERRED FORM'])
uniquePreferredTermsDF.head()

,PREFERRED FORM
0,coal
1,coal mine
2,water
3,liquid water
4,strait


In [7]:
%%time
uniquePreferredTermsDF['Ancestors'] = uniquePreferredTermsDF.apply(lambda x: getAncestors(x['PREFERRED FORM']), axis=1)
uniquePreferredTermsDF['Descendants'] = uniquePreferredTermsDF.apply(lambda x: getDescendants(x['PREFERRED FORM']), axis=1)
print(len(uniquePreferredTermsDF))
uniquePreferredTermsDF.head()


703
CPU times: user 10.7 s, sys: 12.2 ms, total: 10.7 s
Wall time: 10.7 s


,PREFERRED FORM,Ancestors,Descendants
0,coal,"[[ENVO:00001995, ENVO:01000814, BFO:0000040, E...","[[ENVO:2000044, ENVO:2000009, ENVO:2000008, EN..."
1,coal mine,"[[ENVO:01001813, ENVO:00000076, BFO:0000040, B...",[[]]
2,water,"[[CHEBI:33674, CHEBI:37577, BFO:0000001, BFO:0...",[[]]
3,liquid water,"[[ENVO:02000140, BFO:0000040, ENVO:00010483, B...","[[ENVO:01000940, ENVO:01000358, ENVO:00005791,..."
4,strait,"[[ENVO:00000309, BFO:0000040, BFO:0000001, ENV...",[[ENVO:01000651]]


In [8]:
runnerOutputWithAncestors = runnerOutput.merge(uniquePreferredTermsDF, on='PREFERRED FORM', how='left' )
print(len(runnerOutputWithAncestors))
runnerOutputWithAncestors.head()


205959


,DOCUMENT ID,TYPE,MATCHED TERM,PREFERRED FORM,ENTITY ID,Ancestors,Descendants
0,gold:Gb0101224,biolink:OntologyClass,Coal,coal,ENVO:02000091,"[[ENVO:00001995, ENVO:01000814, BFO:0000040, E...","[[ENVO:2000044, ENVO:2000009, ENVO:2000008, EN..."
1,gold:Gb0101224,biolink:OntologyClass,Coal Mine,coal mine,ENVO:00002169,"[[ENVO:01001813, ENVO:00000076, BFO:0000040, B...",[[]]
2,gold:Gb0101224,biolink:ChemicalSubstance,water,water,CHEBI:15377,"[[CHEBI:33674, CHEBI:37577, BFO:0000001, BFO:0...",[[]]
3,gold:Gb0101224,biolink:OntologyClass,water,liquid water,ENVO:00002006_SYNONYM,"[[ENVO:02000140, BFO:0000040, ENVO:00010483, B...","[[ENVO:01000940, ENVO:01000358, ENVO:00005791,..."
4,gold:Gb0101225,biolink:OntologyClass,Coal,coal,ENVO:02000091,"[[ENVO:00001995, ENVO:01000814, BFO:0000040, E...","[[ENVO:2000044, ENVO:2000009, ENVO:2000008, EN..."


In [9]:
goldInput = pd.read_csv(inputDir+inFile, sep='\t', low_memory=False, usecols=['id', 'env_broad_scale', 'env_local_scale', 'env_medium'] )
#goldInput['id'] = goldInput['id'].str.strip('gold:')
goldInput.head()

,id,env_broad_scale,env_local_scale,env_medium
0,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792
1,gold:Gb0101225,ENVO:00002030,ENVO:00002169,ENVO:00005792
2,gold:Gb0101226,ENVO:00002030,ENVO:00002169,ENVO:00005792
3,gold:Gb0101227,ENVO:00002030,ENVO:00002169,ENVO:00005792
4,gold:Gb0101253,ENVO:00000446,ENVO:00002185,ENVO:00002985


In [10]:
inputOutputJoin = goldInput.merge(runnerOutput, left_on='id', right_on='DOCUMENT ID', how='left' )
inputOutputJoin.head()

,id,env_broad_scale,env_local_scale,env_medium,DOCUMENT ID,TYPE,MATCHED TERM,PREFERRED FORM,ENTITY ID
0,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,gold:Gb0101224,biolink:OntologyClass,Coal,coal,ENVO:02000091
1,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,gold:Gb0101224,biolink:OntologyClass,Coal Mine,coal mine,ENVO:00002169
2,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,gold:Gb0101224,biolink:ChemicalSubstance,water,water,CHEBI:15377
3,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,gold:Gb0101224,biolink:OntologyClass,water,liquid water,ENVO:00002006_SYNONYM
4,gold:Gb0101225,ENVO:00002030,ENVO:00002169,ENVO:00005792,gold:Gb0101225,biolink:OntologyClass,Coal,coal,ENVO:02000091


In [11]:
inputOutputJoin = inputOutputJoin.drop(['DOCUMENT ID'], axis=1)
#inputOutputJoin = inputOutputJoin.drop_duplicates()
inputOutputJoin['CURIE'] = inputOutputJoin['ENTITY ID'].str.strip('_SYNONYM')
inputOutputJoin = inputOutputJoin.drop(['ENTITY ID'], axis=1)
inputOutputJoin.head(10)

,id,env_broad_scale,env_local_scale,env_medium,TYPE,MATCHED TERM,PREFERRED FORM,CURIE
0,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,Coal,coal,ENVO:02000091
1,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,Coal Mine,coal mine,ENVO:00002169
2,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:ChemicalSubstance,water,water,CHEBI:15377
3,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,water,liquid water,ENVO:00002006
4,gold:Gb0101225,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,Coal,coal,ENVO:02000091
5,gold:Gb0101225,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,Coal Mine,coal mine,ENVO:00002169
6,gold:Gb0101225,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:ChemicalSubstance,water,water,CHEBI:15377
7,gold:Gb0101225,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,water,liquid water,ENVO:00002006
8,gold:Gb0101226,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,Coal,coal,ENVO:02000091
9,gold:Gb0101226,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,Coal Mine,coal mine,ENVO:00002169


In [12]:
inputOutputJoin.to_csv(outputDir+'inputOutputJoined.tsv', sep='\t', index=None)

In [13]:
filteredMatch = inputOutputJoin.query('CURIE == env_broad_scale' and \
                                      'CURIE == env_local_scale' and \
                                      'CURIE == env_medium')

#filteredMatch = filteredMatch.drop_duplicates()

In [14]:
filteredMatch.head()

,id,env_broad_scale,env_local_scale,env_medium,TYPE,MATCHED TERM,PREFERRED FORM,CURIE
17,gold:Gb0101253,ENVO:00000446,ENVO:00002185,ENVO:00002985,biolink:OntologyClass,oil,oil,ENVO:00002985
24,gold:Gb0101254,ENVO:00000446,ENVO:00002185,ENVO:00002985,biolink:OntologyClass,oil,oil,ENVO:00002985
1454,gold:Gb0102398,ENVO:00000447,ENVO:00000208,ENVO:03000033,biolink:OntologyClass,marine sediment,marine sediment,ENVO:03000033
1460,gold:Gb0102399,ENVO:00000447,ENVO:00000208,ENVO:03000033,biolink:OntologyClass,marine sediment,marine sediment,ENVO:03000033
1466,gold:Gb0102400,ENVO:00000447,ENVO:00000208,ENVO:03000033,biolink:OntologyClass,marine sediment,marine sediment,ENVO:03000033


In [15]:
filteredMatch.to_csv(outputDir+'filteredMatch.tsv', sep='\t', index=None)

In [16]:
len(inputOutputJoin)

208131

In [17]:
# env_{local,medium,broad}_matched
inputOutputJoin['env_broad_matched'] = inputOutputJoin['CURIE'] == inputOutputJoin['env_broad_scale']
inputOutputJoin['env_local_matched'] = inputOutputJoin['CURIE'] == inputOutputJoin['env_local_scale']
inputOutputJoin['env_medium_matched'] = inputOutputJoin['CURIE'] == inputOutputJoin['env_medium']
inputOutputJoin['false_positives'] = (inputOutputJoin.iloc[:,-3:]==False).all(True)

inputOutputJoin.head()

,id,env_broad_scale,env_local_scale,env_medium,TYPE,MATCHED TERM,PREFERRED FORM,CURIE,env_broad_matched,env_local_matched,env_medium_matched,false_positives
0,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,Coal,coal,ENVO:02000091,False,False,False,True
1,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,Coal Mine,coal mine,ENVO:00002169,False,True,False,False
2,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:ChemicalSubstance,water,water,CHEBI:15377,False,False,False,True
3,gold:Gb0101224,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,water,liquid water,ENVO:00002006,False,False,False,True
4,gold:Gb0101225,ENVO:00002030,ENVO:00002169,ENVO:00005792,biolink:OntologyClass,Coal,coal,ENVO:02000091,False,False,False,True


In [18]:
grpList = ['id', 'env_broad_scale', 'env_local_scale', 'env_medium', 'TYPE', 'MATCHED TERM', 'PREFERRED FORM', 'CURIE']

collapsedIO  = inputOutputJoin.groupby(grpList).any()


In [19]:
collapsedIO.head()

env_broad_matched  \
id             env_broad_scale env_local_scale env_medium    TYPE                  MATCHED TERM PREFERRED FORM CURIE                              
gold:Gb0050971 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998              False   
                                                                                   ecosystem    ecosystem      ENVO:01001110              False   
gold:Gb0050972 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998              False   
                                                                                   ecosystem    ecosystem      ENVO:01001110              False   
gold:Gb0050973 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998              False   

                                                                                                                              env_local_matched  \
id             env_broad_scale env_local_scale env_medium    TYPE                  MATCHED TERM PREFERRED FORM CURIE                              
gold:Gb0050971 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998              False   
                                                                                   ecosystem    ecosystem      ENVO:01001110              False   
gold:Gb0050972 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998              False   
                                                                                   ecosystem    ecosystem      ENVO:01001110              False   
gold:Gb0050973 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998              False   

                                                                                                                              env_medium_matched  \
id             env_broad_scale env_local_scale env_medium    TYPE                  MATCHED TERM PREFERRED FORM CURIE                               
gold:Gb0050971 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998                True   
                                                                                   ecosystem    ecosystem      ENVO:01001110               False   
gold:Gb0050972 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998                True   
                                                                                   ecosystem    ecosystem      ENVO:01001110               False   
gold:Gb0050973 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998                True   

                                                                                                                              false_positives  
id             env_broad_scale env_local_scale env_medium    TYPE                  MATCHED TERM PREFERRED FORM CURIE                           
gold:Gb0050971 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998            False  
                                                                                   ecosystem    ecosystem      ENVO:01001110             True  
gold:Gb0050972 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998            False  
                                                                                   ecosystem    ecosystem      ENVO:01001110             True  
gold:Gb0050973 ENVO:00000446   ENVO:01001785   ENVO:00001998 biolink:OntologyClass Soil         soil           ENVO:00001998            False

In [20]:
collapsedIO.to_csv(outputDir+'inputOutputComparison.tsv', sep='\t')

In [21]:
len(collapsedIO)

169239

In [22]:
newDf = pd.read_csv(outputDir+'inputOutputComparison.tsv', sep='\t', low_memory=False)
newDf.head()

,id,env_broad_scale,env_local_scale,env_medium,TYPE,MATCHED TERM,PREFERRED FORM,CURIE,env_broad_matched,env_local_matched,env_medium_matched,false_positives
0,gold:Gb0050971,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,Soil,soil,ENVO:00001998,False,False,True,False
1,gold:Gb0050971,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,ecosystem,ecosystem,ENVO:01001110,False,False,False,True
2,gold:Gb0050972,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,Soil,soil,ENVO:00001998,False,False,True,False
3,gold:Gb0050972,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,ecosystem,ecosystem,ENVO:01001110,False,False,False,True
4,gold:Gb0050973,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,Soil,soil,ENVO:00001998,False,False,True,False


In [23]:
newDf = newDf.merge(uniquePreferredTermsDF, on='PREFERRED FORM', how='left' )
newDf.head()


,id,env_broad_scale,env_local_scale,env_medium,TYPE,MATCHED TERM,PREFERRED FORM,CURIE,env_broad_matched,env_local_matched,env_medium_matched,false_positives,Ancestors,Descendants
0,gold:Gb0050971,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,Soil,soil,ENVO:00001998,False,False,True,False,"[[BFO:0000040, ENVO:00010483, BFO:0000001, BFO...","[[ENVO:00005752, ENVO:00003082, ENVO:00002230,..."
1,gold:Gb0050971,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,ecosystem,ecosystem,ENVO:01001110,False,False,False,True,"[[RO:0002577, ENVO:01000254, BFO:0000040, BFO:...","[[ENVO:01001452, ENVO:00000447, ENVO:00000192,..."
2,gold:Gb0050972,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,Soil,soil,ENVO:00001998,False,False,True,False,"[[BFO:0000040, ENVO:00010483, BFO:0000001, BFO...","[[ENVO:00005752, ENVO:00003082, ENVO:00002230,..."
3,gold:Gb0050972,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,ecosystem,ecosystem,ENVO:01001110,False,False,False,True,"[[RO:0002577, ENVO:01000254, BFO:0000040, BFO:...","[[ENVO:01001452, ENVO:00000447, ENVO:00000192,..."
4,gold:Gb0050973,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,Soil,soil,ENVO:00001998,False,False,True,False,"[[BFO:0000040, ENVO:00010483, BFO:0000001, BFO...","[[ENVO:00005752, ENVO:00003082, ENVO:00002230,..."


In [24]:
newDf.to_csv(outputDir+'inputOutputComparisonWithAncesAndDesc.tsv', sep='\t', index=None)

In [25]:
def checkParentAndDescendant(row, idx):
    #x.env_broad_scale in x.Ancestors[idx]
    if idx == 0:
        if row.env_broad_matched == True:
            return 'ExactMatch'
        else:
            ancestor = row.env_broad_scale in row.Ancestors[0]
            descendant = row.env_broad_scale in row.Descendants[0]
    elif idx == 1:
        if row.env_local_matched == True:
            return 'ExactMatch'
        else:
            ancestor = row.env_local_scale in row.Ancestors[0]
            descendant = row.env_local_scale in row.Descendants[0]
    elif idx == 2:
        if row.env_medium_matched == True:
            return 'ExactMatch'
        else:
            ancestor = row.env_medium in row.Ancestors[0]
            descendant = row.env_medium in row.Descendants[0]
        
    if ancestor:
        return 'AncestorMatch'
    elif descendant:
        return 'DescendantMatch'
    else:
        return 'NoMatch'

In [26]:
%%time
newDf.loc[:,'env_broad_match_type'] = newDf.apply(lambda x:checkParentAndDescendant(x,0) , axis=1)
newDf.loc[:,'env_local_match_type'] = newDf.apply(lambda x:checkParentAndDescendant(x,1) , axis=1)
newDf.loc[:,'env_medium_match_type'] = newDf.apply(lambda x:checkParentAndDescendant(x,2) , axis=1)
newDf.to_csv(outputDir+'inputOutputWithMatches.tsv', sep='\t', index=None)
newDf.head()

CPU times: user 16.4 s, sys: 132 ms, total: 16.6 s
Wall time: 16.6 s


,id,env_broad_scale,env_local_scale,env_medium,TYPE,MATCHED TERM,PREFERRED FORM,CURIE,env_broad_matched,env_local_matched,env_medium_matched,false_positives,Ancestors,Descendants,env_broad_match_type,env_local_match_type,env_medium_match_type
0,gold:Gb0050971,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,Soil,soil,ENVO:00001998,False,False,True,False,"[[BFO:0000040, ENVO:00010483, BFO:0000001, BFO...","[[ENVO:00005752, ENVO:00003082, ENVO:00002230,...",NoMatch,NoMatch,ExactMatch
1,gold:Gb0050971,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,ecosystem,ecosystem,ENVO:01001110,False,False,False,True,"[[RO:0002577, ENVO:01000254, BFO:0000040, BFO:...","[[ENVO:01001452, ENVO:00000447, ENVO:00000192,...",DescendantMatch,NoMatch,NoMatch
2,gold:Gb0050972,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,Soil,soil,ENVO:00001998,False,False,True,False,"[[BFO:0000040, ENVO:00010483, BFO:0000001, BFO...","[[ENVO:00005752, ENVO:00003082, ENVO:00002230,...",NoMatch,NoMatch,ExactMatch
3,gold:Gb0050972,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,ecosystem,ecosystem,ENVO:01001110,False,False,False,True,"[[RO:0002577, ENVO:01000254, BFO:0000040, BFO:...","[[ENVO:01001452, ENVO:00000447, ENVO:00000192,...",DescendantMatch,NoMatch,NoMatch
4,gold:Gb0050973,ENVO:00000446,ENVO:01001785,ENVO:00001998,biolink:OntologyClass,Soil,soil,ENVO:00001998,False,False,True,False,"[[BFO:0000040, ENVO:00010483, BFO:0000001, BFO...","[[ENVO:00005752, ENVO:00003082, ENVO:00002230,...",NoMatch,NoMatch,ExactMatch


In [27]:
def removeDupsInCell(df, col):
    df[col] = df[col].str.split(',').apply(set).str.join(',')
    return df

In [28]:
consolidatedDF = newDf.groupby(['id'])\
                        .agg({
                                'env_broad_scale': lambda a: ','.join(a),
                                'env_local_scale': lambda b: ','.join(b),
                                'env_medium': lambda c: ','.join(c),
                                'TYPE': lambda d: ','.join(d),
                                'MATCHED TERM': lambda e: ','.join(e),
                                'PREFERRED FORM': lambda f: ','.join(f),
                                'CURIE': lambda g: ','.join(g),
                                'env_broad_matched': lambda h: ','.join(str(v).upper() for v in h),
                                'env_local_matched': lambda i: ','.join(str(x).upper() for x in i),
                                'env_medium_matched': lambda j: ','.join(str(y).upper() for y in j),
                                'env_broad_match_type': lambda h: ','.join(v for v in h),
                                'env_local_match_type': lambda i: ','.join(x for x in i),
                                'env_medium_match_type': lambda j: ','.join(y for y in j),
                                'false_positives': lambda k: ','.join(str(z).upper() for z in k),
                                'Ancestors': lambda l: '|'.join(str(u) for u in l),
                                'Descendants': lambda m: '|'.join(str(t) for t in m)
                            })
consolidatedDF.head()

,env_broad_scale,env_local_scale,env_medium,TYPE,MATCHED TERM,PREFERRED FORM,CURIE,env_broad_matched,env_local_matched,env_medium_matched,env_broad_match_type,env_local_match_type,env_medium_match_type,false_positives,Ancestors,Descendants
id,,,,,,,,,,,,,,,,
gold:Gb0050971,"ENVO:00000446,ENVO:00000446","ENVO:01001785,ENVO:01001785","ENVO:00001998,ENVO:00001998","biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110","FALSE,FALSE","FALSE,FALSE","TRUE,FALSE","NoMatch,DescendantMatch","NoMatch,NoMatch","ExactMatch,NoMatch","FALSE,TRUE","[['BFO:0000040', 'ENVO:00010483', 'BFO:0000001...","[['ENVO:00005752', 'ENVO:00003082', 'ENVO:0000..."
gold:Gb0050972,"ENVO:00000446,ENVO:00000446","ENVO:01001785,ENVO:01001785","ENVO:00001998,ENVO:00001998","biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110","FALSE,FALSE","FALSE,FALSE","TRUE,FALSE","NoMatch,DescendantMatch","NoMatch,NoMatch","ExactMatch,NoMatch","FALSE,TRUE","[['BFO:0000040', 'ENVO:00010483', 'BFO:0000001...","[['ENVO:00005752', 'ENVO:00003082', 'ENVO:0000..."
gold:Gb0050973,"ENVO:00000446,ENVO:00000446","ENVO:01001785,ENVO:01001785","ENVO:00001998,ENVO:00001998","biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110","FALSE,FALSE","FALSE,FALSE","TRUE,FALSE","NoMatch,DescendantMatch","NoMatch,NoMatch","ExactMatch,NoMatch","FALSE,TRUE","[['BFO:0000040', 'ENVO:00010483', 'BFO:0000001...","[['ENVO:00005752', 'ENVO:00003082', 'ENVO:0000..."
gold:Gb0050974,"ENVO:00000446,ENVO:00000446","ENVO:01001785,ENVO:01001785","ENVO:00001998,ENVO:00001998","biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110","FALSE,FALSE","FALSE,FALSE","TRUE,FALSE","NoMatch,DescendantMatch","NoMatch,NoMatch","ExactMatch,NoMatch","FALSE,TRUE","[['BFO:0000040', 'ENVO:00010483', 'BFO:0000001...","[['ENVO:00005752', 'ENVO:00003082', 'ENVO:0000..."
gold:Gb0050975,"ENVO:00002030,ENVO:00002030,ENVO:00002030,ENVO...","ENVO:00001997,ENVO:00001997,ENVO:00001997,ENVO...","ENVO:00002034,ENVO:00002034,ENVO:00002034,ENVO...","biolink:ChemicalSubstance,biolink:OntologyClas...","Acid,Acid Mine Drainage,Biofilm,Mine Drainage,...","acid,acid mine drainage,biofilm,mine drainage,...","CHEBI:37527,ENVO:00001997,ENVO:00002034,ENVO:0...","FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALS...","FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE...","FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE...","NoMatch,NoMatch,NoMatch,NoMatch,NoMatch,NoMatc...","NoMatch,ExactMatch,NoMatch,DescendantMatch,NoM...","NoMatch,NoMatch,ExactMatch,NoMatch,NoMatch,NoM...","TRUE,FALSE,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE","[['CHEBI:51086', 'CHEBI:50906']]|[['ENVO:00000...","[['CHEBI:39143', 'CHEBI:48356', 'CHEBI:48360',..."


In [29]:
consolidatedDF = removeDupsInCell(consolidatedDF, 'env_broad_scale')
consolidatedDF = removeDupsInCell(consolidatedDF, 'env_local_scale')
consolidatedDF = removeDupsInCell(consolidatedDF, 'env_medium')

consolidatedDF.head()

,env_broad_scale,env_local_scale,env_medium,TYPE,MATCHED TERM,PREFERRED FORM,CURIE,env_broad_matched,env_local_matched,env_medium_matched,env_broad_match_type,env_local_match_type,env_medium_match_type,false_positives,Ancestors,Descendants
id,,,,,,,,,,,,,,,,
gold:Gb0050971,ENVO:00000446,ENVO:01001785,ENVO:00001998,"biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110","FALSE,FALSE","FALSE,FALSE","TRUE,FALSE","NoMatch,DescendantMatch","NoMatch,NoMatch","ExactMatch,NoMatch","FALSE,TRUE","[['BFO:0000040', 'ENVO:00010483', 'BFO:0000001...","[['ENVO:00005752', 'ENVO:00003082', 'ENVO:0000..."
gold:Gb0050972,ENVO:00000446,ENVO:01001785,ENVO:00001998,"biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110","FALSE,FALSE","FALSE,FALSE","TRUE,FALSE","NoMatch,DescendantMatch","NoMatch,NoMatch","ExactMatch,NoMatch","FALSE,TRUE","[['BFO:0000040', 'ENVO:00010483', 'BFO:0000001...","[['ENVO:00005752', 'ENVO:00003082', 'ENVO:0000..."
gold:Gb0050973,ENVO:00000446,ENVO:01001785,ENVO:00001998,"biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110","FALSE,FALSE","FALSE,FALSE","TRUE,FALSE","NoMatch,DescendantMatch","NoMatch,NoMatch","ExactMatch,NoMatch","FALSE,TRUE","[['BFO:0000040', 'ENVO:00010483', 'BFO:0000001...","[['ENVO:00005752', 'ENVO:00003082', 'ENVO:0000..."
gold:Gb0050974,ENVO:00000446,ENVO:01001785,ENVO:00001998,"biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110","FALSE,FALSE","FALSE,FALSE","TRUE,FALSE","NoMatch,DescendantMatch","NoMatch,NoMatch","ExactMatch,NoMatch","FALSE,TRUE","[['BFO:0000040', 'ENVO:00010483', 'BFO:0000001...","[['ENVO:00005752', 'ENVO:00003082', 'ENVO:0000..."
gold:Gb0050975,ENVO:00002030,ENVO:00001997,ENVO:00002034,"biolink:ChemicalSubstance,biolink:OntologyClas...","Acid,Acid Mine Drainage,Biofilm,Mine Drainage,...","acid,acid mine drainage,biofilm,mine drainage,...","CHEBI:37527,ENVO:00001997,ENVO:00002034,ENVO:0...","FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALS...","FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE...","FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE...","NoMatch,NoMatch,NoMatch,NoMatch,NoMatch,NoMatc...","NoMatch,ExactMatch,NoMatch,DescendantMatch,NoM...","NoMatch,NoMatch,ExactMatch,NoMatch,NoMatch,NoM...","TRUE,FALSE,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE","[['CHEBI:51086', 'CHEBI:50906']]|[['ENVO:00000...","[['CHEBI:39143', 'CHEBI:48356', 'CHEBI:48360',..."


In [30]:
consolidatedDF.to_csv(outputDir+'AllGroupedById.tsv', sep='\t')

In [31]:
newDfSelect = newDf[['id','MATCHED TERM','TYPE', 'PREFERRED FORM', 'CURIE', 'false_positives', 'env_broad_match_type', 'env_local_match_type', 'env_medium_match_type', 'Ancestors', 'Descendants']]
falsePosSubset = newDfSelect[newDfSelect['false_positives'] == True]
falsePosSubset.head()

,id,MATCHED TERM,TYPE,PREFERRED FORM,CURIE,false_positives,env_broad_match_type,env_local_match_type,env_medium_match_type,Ancestors,Descendants
1,gold:Gb0050971,ecosystem,biolink:OntologyClass,ecosystem,ENVO:01001110,True,DescendantMatch,NoMatch,NoMatch,"[[RO:0002577, ENVO:01000254, BFO:0000040, BFO:...","[[ENVO:01001452, ENVO:00000447, ENVO:00000192,..."
3,gold:Gb0050972,ecosystem,biolink:OntologyClass,ecosystem,ENVO:01001110,True,DescendantMatch,NoMatch,NoMatch,"[[RO:0002577, ENVO:01000254, BFO:0000040, BFO:...","[[ENVO:01001452, ENVO:00000447, ENVO:00000192,..."
5,gold:Gb0050973,ecosystem,biolink:OntologyClass,ecosystem,ENVO:01001110,True,DescendantMatch,NoMatch,NoMatch,"[[RO:0002577, ENVO:01000254, BFO:0000040, BFO:...","[[ENVO:01001452, ENVO:00000447, ENVO:00000192,..."
7,gold:Gb0050974,ecosystem,biolink:OntologyClass,ecosystem,ENVO:01001110,True,DescendantMatch,NoMatch,NoMatch,"[[RO:0002577, ENVO:01000254, BFO:0000040, BFO:...","[[ENVO:01001452, ENVO:00000447, ENVO:00000192,..."
8,gold:Gb0050975,Acid,biolink:ChemicalSubstance,acid,CHEBI:37527,True,NoMatch,NoMatch,NoMatch,"[[CHEBI:51086, CHEBI:50906]]","[[CHEBI:39143, CHEBI:48356, CHEBI:48360, CHEBI..."


In [32]:
len(falsePosSubset)

146315

In [33]:
consolidatedFalseCURIEs = falsePosSubset.groupby(['id'])\
                        .agg({
                                'CURIE': lambda x: ','.join(x),
                                'TYPE': lambda a: ','.join(a),
                                'MATCHED TERM': lambda y: ','.join(y),
                                'PREFERRED FORM': lambda z: ','.join(z)
                            })\
                        .rename(columns={'CURIE':'falsePositiveCURIEs'})
consolidatedFalseCURIEs.head(10)

,falsePositiveCURIEs,TYPE,MATCHED TERM,PREFERRED FORM
id,,,,
gold:Gb0050971,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem
gold:Gb0050972,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem
gold:Gb0050973,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem
gold:Gb0050974,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem
gold:Gb0050975,"CHEBI:37527,ENVO:00001996,ENVO:00000083,ENVO:0...","biolink:ChemicalSubstance,biolink:OntologyClas...","Acid,Mine Drainage,Mountain,Mountain,Mountain,...","acid,mine drainage,hill,mount,mountain,seamoun..."
gold:Gb0050977,ENVO:00000015,biolink:OntologyClass,Ocean,ocean
gold:Gb0050978,"ENVO:01001788,ENVO:00000015,ENVO:01001110","biolink:OntologyClass,biolink:OntologyClass,bi...","Marine ecosystem,Ocean,ecosystem","marine ecosystem,ocean,ecosystem"
gold:Gb0050979,"ENVO:00000165,ENVO:00002277,ENVO:00000291,ENVO...","biolink:OntologyClass,biolink:OntologyClass,bi...","Basin,Basin,Basin,Basin,Basin,Basin,Basin,Fall...","asphalt lake,back-arc basin,drainage basin,ket..."
gold:Gb0050980,"UBERON:0001474,UBERON:0002481,ENVO:00000165,EN...","biolink:AnatomicalEntity,biolink:AnatomicalEnt...","Bone,Bone,Basin,Basin,Basin,Basin,Basin,Basin,...","bone element,bone tissue,asphalt lake,back-arc..."


In [34]:
consolidatedFalseCURIEs.to_csv(outputDir+'falsePositivesGroupedById.tsv', sep='\t')

In [35]:
falsePositives = pd.read_csv(outputDir+'falsePositivesGroupedById.tsv', sep='\t', low_memory=False)
falsePositives['false_positives'] = True
falsePositives.head()

,id,falsePositiveCURIEs,TYPE,MATCHED TERM,PREFERRED FORM,false_positives
0,gold:Gb0050971,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem,True
1,gold:Gb0050972,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem,True
2,gold:Gb0050973,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem,True
3,gold:Gb0050974,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem,True
4,gold:Gb0050975,"CHEBI:37527,ENVO:00001996,ENVO:00000083,ENVO:0...","biolink:ChemicalSubstance,biolink:OntologyClas...","Acid,Mine Drainage,Mountain,Mountain,Mountain,...","acid,mine drainage,hill,mount,mountain,seamoun...",True


In [36]:
newDf = newDf.drop(['CURIE', 'TYPE', 'MATCHED TERM', 'PREFERRED FORM'], axis=1)
finalDF = newDf.merge(falsePositives, on=['id', 'false_positives'],how='left')

In [37]:
goldInput = pd.read_csv('../../nlp/data/nlp/input/nlpGOLDInput.tsv', sep='\t', low_memory=False, index_col=None)
goldInput.head()

,id,text
0,gold:Gb0101224,Lithgow State Coal Mine Calcium nutrients (ear...
1,gold:Gb0101225,Lithgow State Coal Mine Calcium nutrients Extr...
2,gold:Gb0101226,Lithgow State Coal Mine Calcium nutrients. Lit...
3,gold:Gb0101227,Lithgow State Coal Mine Calcium merged assembl...
4,gold:Gb0101253,Bass Strait oil field metagenomics - A7A-2. Ba...


In [38]:
finalDF.to_csv(outputDir+'finalOutput.tsv', sep='\t', index=None)

In [39]:
onlyFalsePosivitesDF = newDf[newDf['false_positives'] == True].merge(falsePositives, on='id',how='inner')
onlyFalsePosivitesDF = onlyFalsePosivitesDF.drop(['false_positives_x', 'false_positives_y', 'Ancestors', 'Descendants'], axis=1)
onlyFalsePosivitesDF = onlyFalsePosivitesDF.drop_duplicates()
print(len(onlyFalsePosivitesDF))
onlyFalsePosivitesDF.head()


41725


,id,env_broad_scale,env_local_scale,env_medium,env_broad_matched,env_local_matched,env_medium_matched,env_broad_match_type,env_local_match_type,env_medium_match_type,falsePositiveCURIEs,TYPE,MATCHED TERM,PREFERRED FORM
0,gold:Gb0050971,ENVO:00000446,ENVO:01001785,ENVO:00001998,False,False,False,DescendantMatch,NoMatch,NoMatch,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem
1,gold:Gb0050972,ENVO:00000446,ENVO:01001785,ENVO:00001998,False,False,False,DescendantMatch,NoMatch,NoMatch,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem
2,gold:Gb0050973,ENVO:00000446,ENVO:01001785,ENVO:00001998,False,False,False,DescendantMatch,NoMatch,NoMatch,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem
3,gold:Gb0050974,ENVO:00000446,ENVO:01001785,ENVO:00001998,False,False,False,DescendantMatch,NoMatch,NoMatch,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem
4,gold:Gb0050975,ENVO:00002030,ENVO:00001997,ENVO:00002034,False,False,False,NoMatch,NoMatch,NoMatch,"CHEBI:37527,ENVO:00001996,ENVO:00000083,ENVO:0...","biolink:ChemicalSubstance,biolink:OntologyClas...","Acid,Mine Drainage,Mountain,Mountain,Mountain,...","acid,mine drainage,hill,mount,mountain,seamoun..."


In [40]:
onlyFalsePosWithText = onlyFalsePosivitesDF.merge(goldInput, on='id', how='inner')
onlyFalsePosWithText = onlyFalsePosWithText.drop_duplicates()
onlyFalsePosWithText.head()

,id,env_broad_scale,env_local_scale,env_medium,env_broad_matched,env_local_matched,env_medium_matched,env_broad_match_type,env_local_match_type,env_medium_match_type,falsePositiveCURIEs,TYPE,MATCHED TERM,PREFERRED FORM,text
0,gold:Gb0050971,ENVO:00000446,ENVO:01001785,ENVO:00001998,False,False,False,DescendantMatch,NoMatch,NoMatch,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem,Soil ecosystem from different sites within th...
1,gold:Gb0050972,ENVO:00000446,ENVO:01001785,ENVO:00001998,False,False,False,DescendantMatch,NoMatch,NoMatch,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem,Soil ecosystem from different sites within th...
2,gold:Gb0050973,ENVO:00000446,ENVO:01001785,ENVO:00001998,False,False,False,DescendantMatch,NoMatch,NoMatch,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem,Soil ecosystem from different sites within th...
3,gold:Gb0050974,ENVO:00000446,ENVO:01001785,ENVO:00001998,False,False,False,DescendantMatch,NoMatch,NoMatch,ENVO:01001110,biolink:OntologyClass,ecosystem,ecosystem,Soil ecosystem from different sites within th...
4,gold:Gb0050975,ENVO:00002030,ENVO:00001997,ENVO:00002034,False,False,False,NoMatch,NoMatch,NoMatch,"CHEBI:37527,ENVO:00001996,ENVO:00000083,ENVO:0...","biolink:ChemicalSubstance,biolink:OntologyClas...","Acid,Mine Drainage,Mountain,Mountain,Mountain,...","acid,mine drainage,hill,mount,mountain,seamoun...",Acid Mine Drainage (ARMAN) microbial communiti...


In [41]:
onlyFalsePosWithText.to_csv(outputDir+'onlyFalsePosWithText.tsv', sep='\t', index=None)

In [42]:
allDf = pd.read_csv(outputDir+'AllGroupedById.tsv', sep='\t', low_memory=False)
allDf.head()

,id,env_broad_scale,env_local_scale,env_medium,TYPE,MATCHED TERM,PREFERRED FORM,CURIE,env_broad_matched,env_local_matched,env_medium_matched,env_broad_match_type,env_local_match_type,env_medium_match_type,false_positives,Ancestors,Descendants
0,gold:Gb0050971,ENVO:00000446,ENVO:01001785,ENVO:00001998,"biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110","FALSE,FALSE","FALSE,FALSE","TRUE,FALSE","NoMatch,DescendantMatch","NoMatch,NoMatch","ExactMatch,NoMatch","FALSE,TRUE","[['BFO:0000040', 'ENVO:00010483', 'BFO:0000001...","[['ENVO:00005752', 'ENVO:00003082', 'ENVO:0000..."
1,gold:Gb0050972,ENVO:00000446,ENVO:01001785,ENVO:00001998,"biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110","FALSE,FALSE","FALSE,FALSE","TRUE,FALSE","NoMatch,DescendantMatch","NoMatch,NoMatch","ExactMatch,NoMatch","FALSE,TRUE","[['BFO:0000040', 'ENVO:00010483', 'BFO:0000001...","[['ENVO:00005752', 'ENVO:00003082', 'ENVO:0000..."
2,gold:Gb0050973,ENVO:00000446,ENVO:01001785,ENVO:00001998,"biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110","FALSE,FALSE","FALSE,FALSE","TRUE,FALSE","NoMatch,DescendantMatch","NoMatch,NoMatch","ExactMatch,NoMatch","FALSE,TRUE","[['BFO:0000040', 'ENVO:00010483', 'BFO:0000001...","[['ENVO:00005752', 'ENVO:00003082', 'ENVO:0000..."
3,gold:Gb0050974,ENVO:00000446,ENVO:01001785,ENVO:00001998,"biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem","ENVO:00001998,ENVO:01001110","FALSE,FALSE","FALSE,FALSE","TRUE,FALSE","NoMatch,DescendantMatch","NoMatch,NoMatch","ExactMatch,NoMatch","FALSE,TRUE","[['BFO:0000040', 'ENVO:00010483', 'BFO:0000001...","[['ENVO:00005752', 'ENVO:00003082', 'ENVO:0000..."
4,gold:Gb0050975,ENVO:00002030,ENVO:00001997,ENVO:00002034,"biolink:ChemicalSubstance,biolink:OntologyClas...","Acid,Acid Mine Drainage,Biofilm,Mine Drainage,...","acid,acid mine drainage,biofilm,mine drainage,...","CHEBI:37527,ENVO:00001997,ENVO:00002034,ENVO:0...","FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALS...","FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE...","FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE...","NoMatch,NoMatch,NoMatch,NoMatch,NoMatch,NoMatc...","NoMatch,ExactMatch,NoMatch,DescendantMatch,NoM...","NoMatch,NoMatch,ExactMatch,NoMatch,NoMatch,NoM...","TRUE,FALSE,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE","[['CHEBI:51086', 'CHEBI:50906']]|[['ENVO:00000...","[['CHEBI:39143', 'CHEBI:48356', 'CHEBI:48360',..."


In [43]:
allWithTextDF = allDf.merge(goldInput, on='id', how='inner')
allWithTextDF.insert(2,'env_broad_label',allWithTextDF['env_broad_scale'].apply(lambda x: ont.label(x)))
allWithTextDF.insert(4,'env_local_label',allWithTextDF['env_local_scale'].apply(lambda x: ont.label(x)))
allWithTextDF.insert(6,'env_medium_label',allWithTextDF['env_medium'].apply(lambda x: ont.label(x)))
allWithTextDF.head()

,id,env_broad_scale,env_broad_label,env_local_scale,env_local_label,env_medium,env_medium_label,TYPE,MATCHED TERM,PREFERRED FORM,...,env_broad_matched,env_local_matched,env_medium_matched,env_broad_match_type,env_local_match_type,env_medium_match_type,false_positives,Ancestors,Descendants,text
0,gold:Gb0050971,ENVO:00000446,terrestrial biome,ENVO:01001785,land,ENVO:00001998,soil,"biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem",...,"FALSE,FALSE","FALSE,FALSE","TRUE,FALSE","NoMatch,DescendantMatch","NoMatch,NoMatch","ExactMatch,NoMatch","FALSE,TRUE","[['BFO:0000040', 'ENVO:00010483', 'BFO:0000001...","[['ENVO:00005752', 'ENVO:00003082', 'ENVO:0000...",Soil ecosystem from different sites within th...
1,gold:Gb0050972,ENVO:00000446,terrestrial biome,ENVO:01001785,land,ENVO:00001998,soil,"biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem",...,"FALSE,FALSE","FALSE,FALSE","TRUE,FALSE","NoMatch,DescendantMatch","NoMatch,NoMatch","ExactMatch,NoMatch","FALSE,TRUE","[['BFO:0000040', 'ENVO:00010483', 'BFO:0000001...","[['ENVO:00005752', 'ENVO:00003082', 'ENVO:0000...",Soil ecosystem from different sites within th...
2,gold:Gb0050973,ENVO:00000446,terrestrial biome,ENVO:01001785,land,ENVO:00001998,soil,"biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem",...,"FALSE,FALSE","FALSE,FALSE","TRUE,FALSE","NoMatch,DescendantMatch","NoMatch,NoMatch","ExactMatch,NoMatch","FALSE,TRUE","[['BFO:0000040', 'ENVO:00010483', 'BFO:0000001...","[['ENVO:00005752', 'ENVO:00003082', 'ENVO:0000...",Soil ecosystem from different sites within th...
3,gold:Gb0050974,ENVO:00000446,terrestrial biome,ENVO:01001785,land,ENVO:00001998,soil,"biolink:OntologyClass,biolink:OntologyClass","Soil,ecosystem","soil,ecosystem",...,"FALSE,FALSE","FALSE,FALSE","TRUE,FALSE","NoMatch,DescendantMatch","NoMatch,NoMatch","ExactMatch,NoMatch","FALSE,TRUE","[['BFO:0000040', 'ENVO:00010483', 'BFO:0000001...","[['ENVO:00005752', 'ENVO:00003082', 'ENVO:0000...",Soil ecosystem from different sites within th...
4,gold:Gb0050975,ENVO:00002030,aquatic biome,ENVO:00001997,acid mine drainage,ENVO:00002034,biofilm,"biolink:ChemicalSubstance,biolink:OntologyClas...","Acid,Acid Mine Drainage,Biofilm,Mine Drainage,...","acid,acid mine drainage,biofilm,mine drainage,...",...,"FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALS...","FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE...","FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE...","NoMatch,NoMatch,NoMatch,NoMatch,NoMatch,NoMatc...","NoMatch,ExactMatch,NoMatch,DescendantMatch,NoM...","NoMatch,NoMatch,ExactMatch,NoMatch,NoMatch,NoM...","TRUE,FALSE,FALSE,TRUE,TRUE,TRUE,TRUE,TRUE,TRUE","[['CHEBI:51086', 'CHEBI:50906']]|[['ENVO:00000...","[['CHEBI:39143', 'CHEBI:48356', 'CHEBI:48360',...",Acid Mine Drainage (ARMAN) microbial communiti...


In [44]:
allWithTextDF.to_csv(outputDir+'AllWithText.tsv', sep='\t', index=None)